<p align="center">
<img src="https://mistral.ai/images/logo_hubc88c4ece131b91c7cb753f40e9e1cc5_2589_256x0_resize_q97_h2_lanczos_3.webp" width="100"/>
<img src="https://cdn.iconscout.com/icon/free/png-256/free-python-3521655-2945099.png?f=webp&w=256" width="35"/>
</p>

<h1 align="center">Génération d'un Dataset  pour proceder au fine-tuning d'un modèle Mistral.ai</h1>

---
<p style='text-align: justify;'>
Ce notebook est un outil exploratoire de génération d'un dataset pour proceder au fine-tuning d'un modèle Mistral.ai. Il permet de générer un grand nombre de dialogue sur base d'un prompt. Ce prompt est construit sur base d'une question et d'une réponse. Le prompt est ensuite utilisé pour générer un grand nombre de dialogue. Ces dialogues sont ensuite stockés dans un fichier json. Ce fichier json est ensuite utilisé pour proceder au fine-tuning d'un modèle Mistral.ai.

Dans cet exemple, nous allons générer un dataset pour un CV. La question est "Comment je m'appelle" et la réponse est "Tu t'appelles [nom] [prénom]". Le prompt est ensuite utilisé pour générer 50 variations de la question et de la réponse. Le but est de montrer la capacité du modèle à générer un dataset de qualité de dialogue sur base d'un prompt.
</p>

---

## Installation et importation des données du CV

Ce morceaux de code permet de charger et de pouvoir naviguer dans les données disponibles dans le json

In [23]:
import json

# Chemin vers le fichier JSON
chemin_fichier = 'data.json'

# Ouvrir et lire le fichier JSON
with open(chemin_fichier, 'r') as fichier:
    CV_data = json.load(fichier)

question = ["Comment je m'appelle", "Quand suis je né", "Quel est ma nationnalité", f"""Quel est l'adresse de {CV_data[0]["Identite"]["Prenom"] + ' ' + CV_data[0]["Identite"]["Nom"]}""", 
            f"""Quel est le numéro de téléphone de {CV_data[0]["Identite"]["Prenom"] + ' ' + CV_data[0]["Identite"]["Nom"]}""", f"""Quel est l'adresse e-mail de {CV_data[0]["Identite"]["Prenom"] + ' ' + CV_data[0]["Identite"]["Nom"]}""", 
            f"""Quel est le lien du profil linkedIn de {CV_data[0]["Identite"]["Prenom"] + ' ' + CV_data[0]["Identite"]["Nom"]}""", "Quel est ma dernière expérience professionnelle", "Combien de poste ai-je occupé par le passé", "Quel langue je parle",
            "Combien de langue je parle", "Quel est mon niveau d'anglais", "quel est mon niveau de Français"]
reponse = [f"""Tu t\'appelles {CV_data[0]["Identite"]["Prenom"] + ' ' + CV_data[0]["Identite"]["Nom"]}""", f"""tu es né le {CV_data[0]["Identite"]["Date de naissance"]}""", 
           f"""Tu es de nationalité {CV_data[0]["Identite"]["Nationalite"]}""", f"""Tu habites au {CV_data[0]["Identite"]["Adresse"]}""", f"""Tu peux me joindre au {CV_data[0]["Identite"]["Numero de telephone"]}""",
           f"""Tu peux me contacter par e-mail à l\'adresse {CV_data[0]["Identite"]["Adresse e-mail"]}""", f"""Tu peux consulter mon profil LinkedIn à l\'adresse suivante {CV_data[0]["Identite"]["LinkedIn"]}""",
           f"""Ma dernière expérience professionnelle a été en tant que {list(CV_data[0]["Experiences Professionelles"])[0]} chez {CV_data[0]["Experiences Professionelles"][list(CV_data[0]["Experiences Professionelles"])[0]]["company"]}""",
           f"""J\'ai occupé un total de {len(CV_data[0]["Experiences Professionelles"])} postes au cours de ma carrière.""", f"""Je parle {len(CV_data[0]["LANGUES"])} langues, dont {", ".join(CV_data[0]["LANGUES"])}""", f"""je parle {len(CV_data[0]["LANGUES"])} langues""",
           f"""Mon niveau d\'anglais est {CV_data[0]["LANGUES"]["English"]["level"]}""", f"""Mon niveau de Français est {CV_data[0]["LANGUES"]["French"]["level"]}"""]

question[3]

"Quel est l'adresse de Jean-Baptiste ASTRUZ"

## Création d'un prompt permettant de demander la génération d'un nombre arbitrairement grand de dialogue

Ce prompt à pour but de générer un ordre dynamique qui puisse permettre de générer un grand nombre de réponse sur base d'un grand nombre de questions

In [24]:
prompt = [None] * len(question)
for i in range(len(question)):
    Q = question[i]
    R = reponse[i]
    prompt[i] = f"""Je souhaite que tu me génères exactement 50 variations de la question {Q}?. utilise des éot comme "moi", "mon". De plus, je souhaite que pour chaque variations, tu puisses donner la paraphrase de la réponse qui est "{R}.". N'hesites pas à reformuler la réponse""" + """
    Génère la réponse sous forme de JSON uniquement avec la structure suivante:
    {
        "messages": [
            {
            "role": "user",
            "content" : "Variation 1 de la question"
            },
            {
            "role" : "assistant",
            "content" : "Variation 1 de la réponse"
            },
            {
            "role": "user",
            "content" : "Variation 2 de la question"
            },
            {
            "role" : "assistant",
            "content" : "Variation 2 de la réponse"
            },
            {
            "role": "user",
            "content" : "Variation 3 de la question"
            },
            {
            "role" : "assistant",
            "content" : "Variation 3 de la réponse"
            },
            ...
            {
            "role": "user",
            "content" : "Variation 50 de la question"
            },
            {
            "role" : "assistant",
            "content" : "Variation 50 de la réponse"
            }
        ]
    }
    Ne rajoute pas de texte en dehors de la structure JSON. ni avant ni après. Ne met pas de mise en forme markdown. De plus, tu génères EXACTEMENT 50 variations. Tu ne parles qu'en français. je génère pas plus de 50 variations. Ne rajoute pas de texte en dehors de la structure JSON. ni avant ni après. Ne met pas de mise en forme markdown. De plus, tu génères EXACTEMENT 50 variations. Tu ne parles qu'en français. je génère pas plus de 50 variations.
    """

## Installation et importation des packages nécessaires

On récupère les bons packages pour pouvoir faire le travail d'IA

In [25]:
from mistralai.client import MistralClient # Import the MistralClient from the mistralai.client module
from mistralai.models.chat_completion import ChatMessage # Import the ChatMessage from the mistralai.models.chat_completion module
from dotenv import load_dotenv # Import the load_dotenv function from the dotenv module
import os # Import the os module

## Définition du client Mistral.ai et du modèle à utiliser
On défini le client et le modèle d'IA que l'on va utiliser

In [26]:
load_dotenv()
model = "open-mixtral-8x7b"
api_key= os.getenv('MISTRAL_API_KEY')
client = MistralClient(api_key=api_key, timeout=120)
responses = []
messages = []

On execute le prompt pour pouvoir récupérer les exemples de diaolgue fournis en JSON

In [27]:
for i, Pr in enumerate(prompt) :
    messages.append(ChatMessage(role="user", content=Pr))

In [28]:
for i in range(len(messages)) :
    responses.append(client.chat(model=model, messages= [messages[i]], response_format={"type": "json_object"}, temperature=0))

## Enregistrement des résultats
On va simplement enregistrer les résultats pour pouvoir les charger plus tard

In [40]:
concat_json_object = list()
for i in range(len(messages)):
    
    json_text = responses[i].choices[0].message.content

    json_object = json.loads(json_text)
    # Enregistrer les données JSON dans un fichier
    with open(f'json/{question[i].replace(" ", "_")}.json', 'w') as json_file:
        json.dump(json_object, json_file, indent=4)
    
    concat_json_object.insert(0, json_object)

    print(f"Le fichier JSON a été sauvegardé avec succès dans le fichier json/{question[i].replace(" ", "_")}.json.")

with open('json/Total.json', 'w') as json_file:
    json.dump(concat_json_object, json_file, indent=4)

print(f"Le fichier JSON a été sauvegardé avec succès dans le fichier json/Total.json.")

Le fichier JSON a été sauvegardé avec succès dans le fichier json/Comment_je_m'appelle.json.
Le fichier JSON a été sauvegardé avec succès dans le fichier json/Quand_suis_je_né.json.
Le fichier JSON a été sauvegardé avec succès dans le fichier json/Quel_est_ma_nationnalité.json.
Le fichier JSON a été sauvegardé avec succès dans le fichier json/Quel_est_l'adresse_de_Jean-Baptiste_ASTRUZ.json.
Le fichier JSON a été sauvegardé avec succès dans le fichier json/Quel_est_le_numéro_de_téléphone_de_Jean-Baptiste_ASTRUZ.json.
Le fichier JSON a été sauvegardé avec succès dans le fichier json/Quel_est_l'adresse_e-mail_de_Jean-Baptiste_ASTRUZ.json.
Le fichier JSON a été sauvegardé avec succès dans le fichier json/Quel_est_le_lien_du_profil_linkedIn_de_Jean-Baptiste_ASTRUZ.json.
Le fichier JSON a été sauvegardé avec succès dans le fichier json/Quel_est_ma_dernière_expérience_professionnelle.json.
Le fichier JSON a été sauvegardé avec succès dans le fichier json/Combien_de_poste_ai-je_occupé_par_le_p

## Vérification du nombre de messages générés

In [49]:
import json

# Charger les données JSON depuis le fichier
with open('json/Total.json', 'r') as json_file:
    data = json.load(json_file)

# Initialiser un dictionnaire pour stocker le nombre d'occurrences de chaque rôle
role_count = {}

for i in range(12):
    # Parcourir la liste de messages
    for message in data[i]['messages']:
        role = message['role']
        # Si le rôle est déjà dans le dictionnaire, incrémenter le compteur
        if role in role_count:
            role_count[role] += 1
        # Sinon, ajouter le rôle au dictionnaire avec un compteur initial de 1
        else:
            role_count[role] = 1

# Afficher le nombre d'occurrences de chaque rôle
for role, count in role_count.items():
    print(f"Le rôle '{role}' apparaît {count} fois.")

Le rôle 'user' apparaît 274 fois.
Le rôle 'assistant' apparaît 274 fois.


## Conclusion

Dans ce notebook, nous avons utilisé le modèle Mistral.ai pour générer 50 variations d'une question donnée et leurs réponses correspondantes. Nous avons enregistré les résultats dans un fichier JSON pour une utilisation ultérieure.

Le résultat est satisfaisant, et permet de générer un grand nombre de variations d'une question et de ses réponses. Cela peut être utile pour l'entraînement de modèles de dialogue ou pour la génération de données de test. Les prochaines étapes consiste à permettre de générer des variations de plusieurs questions en même temps, et de pouvoir les générer en parallèle pour gagner du temps. il faudra aussi pouvoir générer des variations de questions et de réponses en plusieurs langues et centraliser les résultats dans un même fichier JSON.

Ce notebook peut être utilisé comme point de départ pour explorer davantage les capacités du modèle Mistral.ai et pour générer des données de dialogue de haute qualité pour diverses applications. il faut aussi implementer un systeme pour entrainer l'IA sur base de ces données générées.

<u>nb</u>: Il est compliquer de forcer le nombre de 50 variations. Il est possible que le nombre de variations générées soit inférieur à 50. Il faudra donc faire attention à ce point.